In [1]:
from pynq import Overlay, MMIO
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load the bitstream
ol = Overlay("./design_1.bit")

# Define the BRAM base address and size
bram_base_address = 0xA0000000
bram_size = 1228800  # 1.2MB

# Create an MMIO instance for the BRAM
bram = MMIO(bram_base_address, bram_size)

# Define the image dimensions
image_width = 400
image_height = 384

# Calculate the number of pixels expected
expected_pixel_count = image_width * image_height

# Define possible channel orders
channel_orders = [
    ('r', 'g', 'b'),  # RGB
    ('g', 'r', 'b'),  # GRB
    ('b', 'g', 'r'),  # BGR
    ('b', 'r', 'g'),  # BRG
    ('g', 'b', 'r'),  # GBR
    ('r', 'b', 'g')   # RBG
]

for order in channel_orders:
    # Read the BRAM content and extract the 12-bit pixels
    pixels = []
    for i in range(0, bram_size, 4):
        data = bram.read(i)
        if len(pixels) >= expected_pixel_count:  # Stop reading when we have enough pixels
            break
        # Extract the 12-bit pixel based on the order
        color_map = {'r': (data >> 8) & 0xF, 'g': (data >> 4) & 0xF, 'b': data & 0xF}
        r = (color_map[order[0]] << 4) | color_map[order[0]]
        g = (color_map[order[1]] << 4) | color_map[order[1]]
        b = (color_map[order[2]] << 4) | color_map[order[2]]
        pixels.append((r, g, b))

    # Ensure the number of pixels matches the expected pixel count
    if len(pixels) != expected_pixel_count:
        print(f"Skipping {order} due to pixel count mismatch: {len(pixels)}")
        continue

    # Convert the list of pixels to a NumPy array and reshape it to the original image dimensions
    image_array = np.array(pixels, dtype=np.uint8).reshape((image_height, image_width, 3))

    # Save the image using PIL
    image = Image.fromarray(image_array)
    image.save(f"./captured_image_{order[0]}{order[1]}{order[2]}.png")
    print(f"Saved captured_image_{order[0]}{order[1]}{order[2]}.png")


Saved captured_image_rgb.png
Saved captured_image_grb.png
Saved captured_image_bgr.png
Saved captured_image_brg.png
Saved captured_image_gbr.png
Saved captured_image_rbg.png
